In [ ]:
import numpy as np
from conduction import ConductionND
from conduction import InversionND
from petsc4py import PETSc
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
minX, maxX = 0.0, 1000.0
minY, maxY = 0.0, 1000.0
minZ, maxZ = -35e3, 1000.0
nx, ny, nz = 10, 9, 10
n = nx*ny*nz

mesh = ConductionND((minX, minY, minZ), (maxX, maxY, maxZ), (nx,ny,nz))

# BCs
mesh.boundary_condition('maxZ', 298.0, flux=False)
mesh.boundary_condition('minZ', 0.04, flux=True)

In [ ]:
lithology = np.zeros((nz,ny,nx), dtype='int32')
lithology[:,3:7,:] = 1
lithology[:,7:,:]  = 2

plt.pcolor(lithology[5,:,:])
plt.colorbar()

In [ ]:
inv = InversionND(lithology.flatten(), mesh)

In [ ]:
def forward_model(self, x):
    k_list, H_list = np.array_split(x[:-1], 2)
    q0 = x[-1]
    
    # map to mesh
    k0, H = self.map(k_list, H_list)
    self.mesh.update_properties(k0, H)
    self.mesh.boundary_condition("maxZ", 298.0, flux=False)
    self.mesh.boundary_condition("minZ", q0, flux=True)
    rhs = self.mesh.construct_rhs()
    
    # solve
    T = self.linear_solve(rhs=rhs)
    q = self.heatflux(T, k0)
    
    cost = 0.0
    cost += self.objective_routine(q=q, T=T) # observations
    cost += self.objective_routine(k=k_list, H=H_list, q0=q0) # priors
    return cost

In [ ]:
def adjoint_model(self, x):
    k_list, H_list = np.array_split(x[:-1], 2)
    q0 = x[-1]
    
    # map to mesh
    k0, H = self.map(k_list, H_list)
    self.mesh.update_properties(k0, H)
    self.mesh.boundary_condition("maxZ", 298.0, flux=False)
    self.mesh.boundary_condition("minZ", q0, flux=True)
    rhs = self.mesh.construct_rhs()
    
    # solve
    T = self.linear_solve(rhs=rhs)
    q = self.heatflux(T, k0)
    
    cost = 0.0
    cost += self.objective_routine(q=q, T=T) # observations
    cost += self.objective_routine(k=k_list, H=H_list, q0=q0) # priors
    
    ## AD ##
    dk = np.zeros_like(H)
    dH = np.zeros_like(H)
    dT = np.zeros_like(H)
    dq0 = np.array(0.0)
    
    # priors
    dcdk_list = self.objective_routine_ad(k=k_list)
    dcdH_list = self.objective_routine_ad(H=H_list)
    dcdq0 = self.objective_routine_ad(q0=q0)
    # observations
    dT += self.objective_routine_ad(T=T)
    dq = self.objective_routine_ad(q=q)

    
    dTq, dkq = self.heatflux_ad(dq, q, T, k0)
    dT += dTq
    dk += dkq
    
    # solve
    dA, db = self.linear_solve_ad(T, dT)
    
    dk += dA
    dH += -db
    dz = np.diff(inv.mesh.grid_coords[-1]).mean()
    lowerBC_mask = self.mesh.bc["minZ"]["mask"]
    dH[lowerBC_mask] += db[lowerBC_mask]/dz
    dq0 += np.sum(-db[lowerBC_mask]/dz/inv.ghost_weights[lowerBC_mask])
    
    # pack to lists
    dk_list, dH_list = inv.map_ad(dk, dH)
    dk_list += dcdk_list
    dH_list += dcdH_list
    dq0 += dcdq0
    
    dx = np.hstack([dk_list, dH_list, [dq0]])
    
    return cost, dx

In [ ]:
k = np.array([3.5, 2.0, 3.2])
H = np.array([0.1e-6, 1e-6, 2e-6])
a = np.array([0., 0., 0.])
q0 = 35e-3

x = np.hstack([k, H, [q0]])
dx = x*0.01

# Priors
k_prior = k*1.1
H_prior = H*1.1
a_prior = a*1.1
q0_prior = np.array(30e-3)

sigma_k = k*0.1
sigma_H = H*0.1
sigma_a = a*0.1
sigma_q0 = np.array(5e-3)

In [ ]:
from conduction.inversion import InvObservation, InvPrior

kp = InvPrior(k_prior, sigma_k)
Hp = InvPrior(H_prior, sigma_H)
ap = InvPrior(a_prior, sigma_a)
q0p = InvPrior(q0_prior, sigma_q0)

inv.add_prior(k=kp, H=Hp, q0=q0p)

In [ ]:
np.random.seed(0)

qs = np.ones(5)*0.03
sigma_qs = qs*0.5
qs_coord = np.zeros((5,3))
qs_coord[:,0] = np.random.random(5)*1e3
qs_coord[:,1] = 0.0
qs_coord[:,2] = np.random.random(5)*1e3

qobs = InvObservation(qs, sigma_qs, qs_coord)

inv.add_observation(q=qobs)

In [ ]:
fm0 = forward_model(inv, x)
fm1 = forward_model(inv, x+dx)
ad = adjoint_model(inv, x)

print "finite difference", (fm1 - fm0)
print "adjoint", ad[1].dot(dx)